In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import ipywidgets as widgets
from ipywidgets import VBox, HBox
from IPython.display import display, clear_output

# ---------- Core math (BSM with continuous dividend yield q) ----------

def _d1_d2(S, K, T, r, sigma, q=0.0):
    """Return d1, d2 for (possibly array) inputs with safe guards."""
    S = np.asarray(S, dtype=float)
    K = np.asarray(K, dtype=float)
    T = np.asarray(T, dtype=float)
    sigma = np.asarray(sigma, dtype=float)
    q = float(q)

    epsT = 1e-12
    epsS = 1e-300
    sqrtT = np.sqrt(np.maximum(T, epsT))
    with np.errstate(divide='ignore', invalid='ignore'):
        d1 = (np.log(np.maximum(S, epsS)/np.maximum(K, epsS)) + (r - q + 0.5 * sigma**2) * T) / (sigma * sqrtT)
    d2 = d1 - sigma * sqrtT
    return d1, d2

def bs_option_price(S, K, T, r, sigma, option_type='call', q=0.0):
    """
    Black–Scholes–Merton price with dividend yield q.
    Handles T=0 or sigma close to 0 by returning intrinsic value.
    """
    if T <= 0 or sigma <= 0:
        intrinsic = max(0.0, S - K) if option_type == 'call' else max(0.0, K - S)
        return float(intrinsic)

    d1, d2 = _d1_d2(S, K, T, r, sigma, q)
    if option_type == 'call':
        return float(S * np.exp(-q*T) * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2))
    else:
        return float(K * np.exp(-r*T) * norm.cdf(-d2) - S * np.exp(-q*T) * norm.cdf(-d1))

def bs_greeks(S, K, T, r, sigma, option_type='call', q=0.0, theta_per_day=True, vega_per_1pct=True, rho_per_1pct=True):
    """
    Greeks under BSM with dividend yield q.
    Returns: delta, gamma, vega, theta, rho
    """
    if T <= 0 or sigma <= 0:
        # Reasonable limiting values at expiry/zero vol
        intrinsic = (S - K) if option_type == 'call' else (K - S)
        delta = 1.0 if (option_type == 'call' and S > K) else (-1.0 if (option_type == 'put' and S < K) else 0.0)
        gamma = 0.0
        vega = 0.0
        theta = 0.0
        rho = 0.0
        return delta, gamma, vega, theta, rho

    d1, d2 = _d1_d2(S, K, T, r, sigma, q)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    nd1 = norm.pdf(d1)

    # Delta
    if option_type == 'call':
        delta = np.exp(-q*T) * Nd1
    else:
        delta = np.exp(-q*T) * (Nd1 - 1)

    # Gamma
    gamma = np.exp(-q*T) * nd1 / (S * sigma * np.sqrt(T))

    # Vega (per 1% if requested)
    vega = S * np.exp(-q*T) * nd1 * np.sqrt(T)
    if vega_per_1pct:
        vega /= 100.0

    # Theta (per year then optionally per day)
    # Call theta:
    theta_call = (-S*np.exp(-q*T)*nd1*sigma/(2*np.sqrt(T))
                  + q*S*np.exp(-q*T)*Nd1
                  - r*K*np.exp(-r*T)*Nd2)
    # Put theta:
    theta_put  = (-S*np.exp(-q*T)*nd1*sigma/(2*np.sqrt(T))
                  - q*S*np.exp(-q*T)*norm.cdf(-d1)
                  + r*K*np.exp(-r*T)*norm.cdf(-d2))
    theta = theta_call if option_type == 'call' else theta_put
    if theta_per_day:
        theta /= 365.0

    # Rho (per 1% if requested)
    if option_type == 'call':
        rho = K*T*np.exp(-r*T)*Nd2
    else:
        rho = -K*T*np.exp(-r*T)*norm.cdf(-d2)
    if rho_per_1pct:
        rho /= 100.0

    return float(delta), float(gamma), float(vega), float(theta), float(rho)

# ---------- Forwards / Futures ----------
def forward_price(S, r, T):
    return float(S * np.exp(r * T))

def futures_price(S, r, T, d=0.0):
    return float(S * np.exp((r - d) * T))

# ---------- Plotting helpers ----------
def plot_payoff(S, K, premium, option_type, grow_premium=False, r=0.0, T=0.0):
    ST = np.linspace(0.5*S, 1.5*S, 200)
    cost = premium * (np.exp(r*T) if grow_premium else 1.0)
    if option_type == 'call':
        payoff = np.maximum(ST - K, 0) - cost
        breakeven = K + cost
    else:
        payoff = np.maximum(K - ST, 0) - cost
        breakeven = K - cost

    plt.figure(figsize=(8,5))
    plt.plot(ST, payoff, label=f'{option_type.capitalize()} P/L at expiry')
    plt.axhline(0, linestyle='--')
    plt.axvline(K, linestyle='--', label='Strike')
    plt.axvline(breakeven, linestyle=':', label='Breakeven')
    plt.title(f"{option_type.capitalize()} Option Payoff Diagram")
    plt.xlabel("Spot at Expiry (S_T)")
    plt.ylabel("Profit / Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

def plot_greeks_vs_spot(S, K, T, r, sigma, option_type, q=0.0):
    ST = np.linspace(0.5*S, 1.5*S, 200)
    dlt, gmm, vga, tht, rho = [], [], [], [], []
    for s in ST:
        D,G,V,Th,R = bs_greeks(s, K, T, r, sigma, option_type, q=q)
        dlt.append(D); gmm.append(G); vga.append(V); tht.append(Th); rho.append(R)

    fig, axs = plt.subplots(3, 2, figsize=(12,10), sharex=True)
    axs = axs.flatten()

    axs[0].plot(ST, dlt); axs[0].set_title('Delta'); axs[0].grid(True)
    axs[1].plot(ST, gmm); axs[1].set_title('Gamma'); axs[1].grid(True)
    axs[2].plot(ST, vga); axs[2].set_title('Vega (per 1%)'); axs[2].grid(True)
    axs[3].plot(ST, tht); axs[3].set_title('Theta (per day)'); axs[3].grid(True)
    axs[4].plot(ST, rho); axs[4].set_title('Rho (per 1%)'); axs[4].grid(True)

    axs[5].axis('off')  # hide the unused subplot
    for ax in axs[:-1]:
        ax.axvline(K, linestyle='--')
    fig.suptitle("Option Greeks vs Spot", fontsize=16)
    axs[2].set_xlabel("Spot Price (S)")
    axs[3].set_xlabel("Spot Price (S)")
    axs[4].set_xlabel("Spot Price (S)")
    plt.tight_layout()
    plt.show()

def plot_gamma_heatmap(S, K, T, r, sigma, option_type, q=0.0):
    # Vectorized grid
    S_vals = np.linspace(0.5*S, 1.5*S, 100)
    T_vals = np.linspace(max(1e-3, 0.01), max(0.02, T), 100)  # keep >0
    S_grid, T_grid = np.meshgrid(S_vals, T_vals)
    d1, _ = _d1_d2(S_grid, K, T_grid, r, sigma, q)
    gamma = np.exp(-q*T_grid) * norm.pdf(d1) / (S_grid * sigma * np.sqrt(T_grid))

    plt.figure(figsize=(9,6))
    plt.title("Gamma Heatmap")
    plt.xlabel("Spot Price (S)")
    plt.ylabel("Time to Expiry (T, years)")
    cs = plt.contourf(S_grid, T_grid, gamma, levels=20)
    plt.colorbar(cs, label='Gamma')
    plt.show()

# ---------- Dashboard glue ----------
def calculate_all(S, K, T, r, sigma, option_type, q, d, grow_premium):
    clear_output(wait=True)
    print(f"📈 Spot: ₹{S} | 📌 Strike: ₹{K} | ⏳ T: {T}y")
    print(f"🏦 r: {r:.4f} (annual) | 📉 q: {q:.4f} | 📊 σ: {sigma:.4f}")
    print(f"📝 Type: {option_type.capitalize()} | 🛒 Cost-of-capital in P/L: {grow_premium}")
    print("-"*48)

    fwd = forward_price(S, r, T)
    fut = futures_price(S, r, T, d=d)
    price = bs_option_price(S, K, T, r, sigma, option_type, q=q)
    delta, gamma, vega, theta, rho = bs_greeks(S, K, T, r, sigma, option_type, q=q)

    print(f"📍 Forward Price (no income): ₹{fwd:.2f}")
    print(f"📍 Futures Price (carry d={d:.4f}): ₹{fut:.2f}")
    print(f"📍 {option_type.capitalize()} Price (BSM, q={q:.4f}): ₹{price:.2f}")
    print(f"Greeks  Δ: {delta:.4f} | Γ: {gamma:.6f} | Vega(1%): {vega:.4f} | Theta/day: {theta:.4f} | Rho(1%): {rho:.4f}")

    # Checks (optional): put–call parity if you want to compute both sides
    # parity_lhs = price if option_type=='call' else ...
    # parity_rhs = ...

    plot_payoff(S, K, price, option_type, grow_premium=grow_premium, r=r, T=T)
    plot_greeks_vs_spot(S, K, T, r, sigma, option_type, q=q)
    plot_gamma_heatmap(S, K, T, r, sigma, option_type, q=q)

# ---------- Widgets ----------
S_input = widgets.FloatSlider(value=100, min=1, max=1000, step=1, description='Spot S')
K_input = widgets.FloatSlider(value=100, min=1, max=1000, step=1, description='Strike K')
T_input = widgets.FloatSlider(value=0.5, min=0.0, max=5.0, step=0.01, description='Time T (y)')
r_input = widgets.FloatSlider(value=0.05, min=-0.05, max=0.25, step=0.001, description='r (annual)')
q_input = widgets.FloatSlider(value=0.00, min=0.0, max=0.15, step=0.001, description='q (div yld)')
d_input = widgets.FloatSlider(value=0.00, min=-0.05, max=0.15, step=0.001, description='d (carry)')
sigma_input = widgets.FloatSlider(value=0.20, min=0.01, max=2.00, step=0.005, description='σ (annual)')
option_type_input = widgets.ToggleButtons(options=['call', 'put'], description='Type')
grow_premium_input = widgets.Checkbox(value=False, description='Grow premium to expiry')

ui = VBox([
    HBox([S_input, K_input, T_input]),
    HBox([r_input, q_input, d_input, sigma_input]),
    HBox([option_type_input, grow_premium_input])
])

out = widgets.interactive_output(
    calculate_all,
    {
        'S': S_input, 'K': K_input, 'T': T_input, 'r': r_input, 'sigma': sigma_input,
        'option_type': option_type_input, 'q': q_input, 'd': d_input, 'grow_premium': grow_premium_input
    }
)

display(ui, out)


Output()